In [2]:
from saai.multi_nlu_client import nlu_parse
from pprint import pprint
url='http://localhost:15008'
result=await nlu_parse(url, 'how about french food')
pprint(result)

{'entities': [{'confidence': 0.9960352006,
               'end': 16,
               'entity': 'cuisine',
               'extractor': 'CRFEntityExtractor',
               'start': 10,
               'value': 'french'}],
 'intent': {'confidence': 0.9999802113, 'name': 'inform'},
 'intent_ranking': [{'confidence': 0.9999802113, 'name': 'inform'},
                    {'confidence': 6.9449e-06, 'name': 'email_id'},
                    {'confidence': 5.7164e-06, 'name': 'mood_unhappy'},
                    {'confidence': 3.6971e-06, 'name': 'goodbye'},
                    {'confidence': 1.2413e-06, 'name': 'bot_challenge'},
                    {'confidence': 7.163e-07, 'name': 'greet'},
                    {'confidence': 5.65e-07, 'name': 'mood_great'},
                    {'confidence': 3.832e-07, 'name': 'order_now'},
                    {'confidence': 2.54e-07, 'name': 'request_info'},
                    {'confidence': 2.487e-07, 'name': 'thankyou'}],
 'text': 'how about french food'}


In [3]:
result['entities']

[{'start': 10,
  'end': 16,
  'value': 'french',
  'entity': 'cuisine',
  'confidence': 0.9960352006,
  'extractor': 'CRFEntityExtractor'}]

In [20]:
from saai.tool import rasa_nlu_parse, rasa_nlu_vis
# sents='how about street food'
# sents='how about chinese food'
sents='how about north eastern food'

result=rasa_nlu_parse(sents, 'http://localhost:15008')
ents = result['entities']
ents

[{'start': 10,
  'end': 23,
  'value': 'north eastern',
  'entity': 'cuisine',
  'confidence': 0.7435030861,
  'extractor': 'CRFEntityExtractor'}]

In [21]:
rasa_nlu_vis(sents, 'http://localhost:15008')

BoxMarkup('how about north eastern food', [Span(10, 23, 'cuisine')])

In [22]:
from sagas.nlu.ruleset_procs import list_words, cached_chunks, get_main_domains
from sagas.conf.conf import cf
lang='en'
chunks = cached_chunks(sents, lang, cf.engine(lang))
[(w.text, w.entity or 'O') for w in chunks['doc'].words]

[('how', 'O'),
 ('about', 'O'),
 ('north', 'O'),
 ('eastern', 'O'),
 ('food', 'O'),
 ('.', 'O')]

In [10]:
# sents.index('xx', 0)

In [23]:
def set_word_positions(doc):
    running_offset = 0
    rs = []
    pos_map={}
    for token in doc.words:
        word = token.text    
        word_offset = sents.find(word, running_offset)
        if word_offset>-1:
            word_len = len(word)
            running_offset = word_offset + word_len
            pos_map[token.index]=(word_offset, running_offset)
    return pos_map

pos_map=set_word_positions(chunks['doc'])
print([(w.text, w.index) for w in chunks['doc'].words])
pos_map

[('how', '1'), ('about', '2'), ('north', '3'), ('eastern', '4'), ('food', '5'), ('.', '6')]


{'1': (0, 3), '2': (4, 9), '3': (10, 15), '4': (16, 23), '5': (24, 28)}

In [24]:
running_offset = 0
rs = []
for token in chunks['doc'].words:
    word = token.text    
    word_offset = sents.find(word, running_offset)
    if word_offset>-1:
        word_len = len(word)
        running_offset = word_offset + word_len
        print(f"{word} - ({word_offset}, {running_offset})")
        for ent in ents:
            start, end=ent['start'], ent['end']
            if word_offset>=start and running_offset<=end:
                rs.append({"start": word_offset,
                           "end": running_offset,
                           'value': word,
                           'entity': ent['entity']
                           })
rs            

how - (0, 3)
about - (4, 9)
north - (10, 15)
eastern - (16, 23)
food - (24, 28)


[{'start': 10, 'end': 15, 'value': 'north', 'entity': 'cuisine'},
 {'start': 16, 'end': 23, 'value': 'eastern', 'entity': 'cuisine'}]

In [25]:
for ent in ents:
    word=ent['value']
    word_offset = sents.index(word, ent['start'])
    print(word_offset)

10
